# NAMED ENTITY RECOGNITION:

1. The named entities are pre-defined categories chosen according to the use case such as names of people, organizations, places, codes, time notations, monetary values, etc.

1. NER aims to assign a class to each token (usually a single word) in a sequence. Because of this, NER is also referred to as token classification.

In [94]:
!pip install simpletransformers

In [95]:
import pandas as pd
data = pd.read_csv("Sentense.csv",encoding="latin1" )

In [96]:
data.head(30)

,Sentence #,words,Tag
0,Sentence 0,the,o
1,NaN,export,o
2,NaN,of,o
3,NaN,Transport,B-Product
4,NaN,Equipment,I-Product
5,NaN,and,O
6,NaN,Parts,B-Product
7,NaN,Thereof,I-Product
8,NaN,to,o
9,NaN,POLAND,B-Country


In [97]:
data =data.fillna(method ="ffill")

In [98]:
data.head(30)

,Sentence #,words,Tag
0,Sentence 0,the,o
1,Sentence 0,export,o
2,Sentence 0,of,o
3,Sentence 0,Transport,B-Product
4,Sentence 0,Equipment,I-Product
5,Sentence 0,and,O
6,Sentence 0,Parts,B-Product
7,Sentence 0,Thereof,I-Product
8,Sentence 0,to,o
9,Sentence 0,POLAND,B-Country


In [99]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [100]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

In [101]:
data.head(30)

,Sentence #,words,Tag
0,0,the,o
1,0,export,o
2,0,of,o
3,0,Transport,B-Product
4,0,Equipment,I-Product
5,0,and,O
6,0,Parts,B-Product
7,0,Thereof,I-Product
8,0,to,o
9,0,POLAND,B-Country


In [102]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [103]:
data["labels"] = data["labels"].str.upper()

In [104]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [105]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [106]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [107]:
train_data

,sentence_id,words,labels
13181,7079,on,O
153077,7895,Products,I-PRODUCT
36126,1137,BAHRAIN,B-COUNTRY
139027,6805,was,O
56364,2156,Asbestos,B-PRODUCT
...,...,...,...
146542,7349,the,O
170720,8873,Appliances,I-PRODUCT
75129,3131,of,O
108224,4934,export,O


# Model Training


In [108]:
from simpletransformers.ner import NERModel,NERArgs

In [109]:
label = data["labels"].unique().tolist()
label

['O',
 'B-PRODUCT',
 'I-PRODUCT',
 'B-COUNTRY',
 'B-TIME',
 'I-TIME',
 'B-CURRUNCY',
 'I-COUNTRY']

In [110]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32


In [111]:
model = NERModel('roberta', 'roberta-base',labels=label,args =args)


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [112]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/313 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


(313, 0.2249546022698902)

In [113]:
result, model_outputs, preds_list = model.eval_model(test_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/307 [00:00<?, ?it/s]

In [114]:
result

{'eval_loss': 0.2232609057765043,
 'precision': 0.8951743495627766,
 'recall': 0.8988130724621972,
 'f1_score': 0.8969900208237553}

In [121]:
prediction, model_output = model.predict(["how much the Export to jordan in December 2020 "])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [122]:
prediction

[[{'how': 'O'},
  {'much': 'O'},
  {'the': 'O'},
  {'Export': 'I-PRODUCT'},
  {'to': 'O'},
  {'jordan': 'B-COUNTRY'},
  {'in': 'O'},
  {'December': 'I-PRODUCT'},
  {'2020': 'I-TIME'}]]